In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif 
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score 
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate


from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif
#import shap

gpu_devices = tf.config.experimental.list_physical_devices('GPU')

if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')
    
    
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



Shutdown_Physiologic = pd.read_csv("Shutdown_Physiologic.csv").drop('Unnamed: 0', axis=1).set_index(['case_id','phenotype'])

cat_encoder = OneHotEncoder(sparse=False)

pheno_type_cat = Shutdown_Physiologic[['pheno']]
pheno_type_cat_1hot = cat_encoder.fit_transform(pheno_type_cat)
Shutdown_Physiologic['pheno'] = pheno_type_cat_1hot

#scaled_Shutdown_Physiologic = (Shutdown_Physiologic - Shutdown_Physiologic.mean()) / Shutdown_Physiologic.std()
sklr = StandardScaler()

y = Shutdown_Physiologic['pheno']
y = y.astype(int)

X_to_drop = Shutdown_Physiologic.drop("pheno", axis=1)
temp_X = sklr.fit_transform(X_to_drop)
X = pd.DataFrame(temp_X,columns=X_to_drop.columns, index = X_to_drop.index)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y)

feats_list = []



Using GPU


In [76]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

xgb_clf = XGBClassifier(random_state=42)
Ada_clf = AdaBoostClassifier(xgb_clf, random_state=42)

accuracy_scoreDict = {'Classifiers': [], "BestScore": [], "BestEstimator" : []}

Parameters_df = pd.DataFrame(columns=['Classifier', 'Parameters'])

names = [
    "XGB",
    "ADABoost"
    "SVC",
    "LogReg",    
    "Random Forest",
    "Neural Net",
    "Naive Bayes"
]

classifiers = [
    XGBClassifier(random_state=42),
    AdaBoostClassifier(random_state=42),

    SVC(random_state=1),
    LogisticRegression(random_state=1),
    RandomForestClassifier(random_state=1),
    MLPClassifier(random_state=1),
]


param_grid = [ 
    {'learning_rate': [0.001, 0.01, .1], 'booster':['gbtree', 'gblinear', 'dart'], 'gamma':[0.5, 1.0, 1.5]},
    {'n_estimators': [25, 75, 110], 'learning_rate':[0.5, 1.0, 1.5],'algorithm':['SAMME', 'SAMME.R']},
    {'C': [0.5, 1.0, 1.5], 'kernel': ['poly', 'rbf', 'sigmoid'], 'max_iter':[500, 1250, 2000]},
    {'C': [0.5, 1.0, 1.5], 'penalty': ['l1', 'l2', 'elasticnet', 'none']},
    {'n_estimators': [50, 125, 250],'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [None, 5, 20]},
    {'hidden_layer_sizes': [50, 100, 125], 'activation': ['identity', 'logistic', 'relu'], 'alpha': [.0001, .001, .01]},
    ]


  

for clf, param_grid in zip(classifiers, param_grid):
    grid_search = GridSearchCV(clf, cv=5,param_grid=param_grid , scoring='accuracy', return_train_score=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11, test_size=.20)

  
    grid_search.fit(X_train, y_train)
    #score = grid_search.score(X_test, y_test)
    #print("Test Score: {} With Classifier: {}".format(score, clf))
    # Get the classifier's parameters
    params = clf.get_params()
    # Add the classifier name and parameters to the dataframe
    Parameters_df = Parameters_df.append({'Classifier': clf.__class__.__name__, 'Parameters': params}, ignore_index=True)

    
    #accuracy_scoreDict['Accuracy Score'].append(score) 
    accuracy_scoreDict['Classifiers'].append(clf.__class__.__name__)
    accuracy_scoreDict['BestScore'].append(grid_search.best_score_)
    accuracy_scoreDict['BestEstimator'].append(grid_search.best_estimator_)
    print("-"*90)
    print(f"Best hyperparameters for {clf.__class__.__name__}: {grid_search.best_params_}")
    print(f"Best score for {clf.__class__.__name__}: {grid_search.best_score_:.3f}")
    print("-"*90)
          

accuracy_score_df = pd.DataFrame(accuracy_scoreDict)

accuracy_score_df['Parameters'] = Parameters_df.Parameters


------------------------------------------------------------------------------------------
Best hyperparameters for XGBClassifier: {'booster': 'gblinear', 'gamma': 0.5, 'learning_rate': 0.01}
Best score for XGBClassifier: 0.803
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Best hyperparameters for AdaBoostClassifier: {'algorithm': 'SAMME', 'learning_rate': 1.5, 'n_estimators': 75}
Best score for AdaBoostClassifier: 0.806
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Best hyperparameters for SVC: {'C': 1.0, 'kernel': 'sigmoid', 'max_iter': 500}
Best score for SVC: 0.778
------------------------------------------------------------------------------------------
-------------------------------------------------------------

In [77]:
accuracy_score_df

,Classifiers,BestScore,BestEstimator,Parameters
0,XGBClassifier,0.802778,"XGBClassifier(booster='gblinear', gamma=0.5, l...","{'base_score': 0.5, 'booster': 'gbtree', 'cols..."
1,AdaBoostClassifier,0.805556,"(DecisionTreeClassifier(max_depth=1, random_st...","{'algorithm': 'SAMME.R', 'base_estimator': Non..."
2,SVC,0.777778,"SVC(kernel='sigmoid', max_iter=500, random_sta...","{'C': 1.0, 'break_ties': False, 'cache_size': ..."
3,LogisticRegression,0.805556,"LogisticRegression(C=0.5, penalty='none', rand...","{'C': 1.0, 'class_weight': None, 'dual': False..."
4,RandomForestClassifier,0.733333,"(DecisionTreeClassifier(max_features='auto', r...","{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
5,MLPClassifier,0.802778,"MLPClassifier(activation='logistic', hidden_la...","{'activation': 'relu', 'alpha': 0.0001, 'batch..."


In [72]:
from sklearn.model_selection import GridSearchCV

svcclf = SVC(kernel='sigmoid', max_iter=500, random_state=1)
logregclf = LogisticRegression(C=0.5, penalty='none', random_state=1)
rf_clf = RandomForestClassifier(n_estimators=125, random_state=1)
mlpclf = MLPClassifier(activation='logistic', hidden_layer_sizes=100, random_state=1)
voting_clf = VotingClassifier(estimators=[('lr', logregclf), ('rf', rf_clf), ('svc', svcclf), ('mlp', mlpclf)], voting='hard')
scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro'}

  
for clf in (logregclf, rf_clf, svcclf, mlpclf, voting_clf):
  scores = cross_validate(clf, X, y, scoring=scoring,
                         cv=10, return_train_score=True)
  scoreKeys = pd.DataFrame(scores)
  print("-"*90)
  print(f"Cross Validaion Metrics for {clf.__class__.__name__}:")
  print(scoreKeys[['test_acc','test_prec_macro','test_rec_micro']])
  print("-"*90)
  

------------------------------------------------------------------------------------------
Cross Validaion Metrics for LogisticRegression:
   test_acc  test_prec_macro  test_rec_micro
0  0.666667         0.800000        0.666667
1  1.000000         1.000000        1.000000
2  0.800000         0.833333        0.833333
3  0.600000         0.583333        0.583333
4  0.400000         0.250000        0.333333
5  0.800000         0.833333        0.833333
6  0.800000         0.833333        0.833333
7  0.800000         0.833333        0.833333
8  0.800000         0.833333        0.833333
9  0.400000         0.416667        0.416667
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Cross Validaion Metrics for RandomForestClassifier:
   test_acc  test_prec_macro  test_rec_micro
0  0.500000         0.500000        0.500000
1  0.833333         0.875000        0.8333

In [68]:
scores

,fit_time,score_time,test_acc,train_acc,test_prec_macro,train_prec_macro,test_rec_micro,train_rec_micro
0,0.519785,0.036659,0.666667,1.0,0.800000,1.0,0.666667,1.0
1,0.514792,0.032379,0.833333,1.0,0.875000,1.0,0.833333,1.0
2,0.500297,0.031802,0.800000,1.0,0.833333,1.0,0.833333,1.0
3,0.513004,0.031779,0.600000,1.0,0.583333,1.0,0.583333,1.0
4,0.497285,0.036940,0.600000,1.0,0.583333,1.0,0.583333,1.0
5,0.512389,0.031775,1.000000,1.0,1.000000,1.0,1.000000,1.0
6,0.493506,0.031688,0.800000,1.0,0.833333,1.0,0.833333,1.0
7,0.488975,0.031585,1.000000,1.0,1.000000,1.0,1.000000,1.0
8,0.528861,0.031790,0.600000,1.0,0.750000,1.0,0.666667,1.0
9,0.496676,0.034509,0.400000,1.0,0.416667,1.0,0.416667,1.0


In [33]:

#feature_selectors = ['chi2','mutual_info_classif','SelectFromModel']

import pandas as pd

# Create an empty dataframe
df = pd.DataFrame(columns=['Classifier', 'Parameters'])

# Iterate over the list of classifiers
for name, clf in zip(names, classifiers):
  
# Display the resulting dataframe
df


,Classifier,Parameters
0,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ..."
1,LogReg,"{'C': 1.0, 'class_weight': None, 'dual': False..."
2,Random Forest,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
3,Neural Net,"{'activation': 'relu', 'alpha': 0.0001, 'batch..."


In [ ]:
param_grid = [ 
    {'C': [0.5, 1.0, 1.5], 'kernel': ['poly', 'rbf', 'sigmoid'], 'max_iter':[500, 1250, 2000]},
    {'C': [0.5, 1.0, 1.5], 'solver': ['lbfgs','liblinear','saga']},
    {'n_estimators': [50, 125, 250],'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [None, 5, 20]},
    {'hidden_layer_sizes': [50, 100, 125], 'activation': ['identity', 'logistic', 'relu'], 'alpha': [.0001, .001, .01]},
    
    ]


In [ ]:
from sklearn.model_selection import GridSearchCV

names = [
    "SVC",
    "LogReg",    
    "Random Forest",
    "Neural Net",
    "Naive Bayes"
]

classifiers = [
    SVC(random_state=1),
    LogisticRegression(random_state=1),
    RandomForestClassifier(random_state=1),
    MLPClassifier(random_state=1),
]

param_grid = [ 
    {'C': [0.5, 1.0, 1.5], 'kernel': ['poly', 'rbf', 'sigmoid'], 'max_iter':[500, 1250, 2000]},
    {'penalty': ['l1', 'l2','elasticnet'], 'C': [0.5, 1.0, 1.5], 'solver': ['lbfgs','liblinear','saga']},
    {'n_estimators': [50, 125, 250],'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [None, 5, 20]},
    {'hidden_layer_sizes': [50, 100, 125], 'activation': ['identity', 'logistic', 'relu'], 'alpha': [.0001, .001, .01]},
    
    ]




GridSearchCVresults = {'Classifiers': [], 'Params': [], "BestScore": [], "BestEstimator" : []}


for clf, params in zip(classifiers, param_grid):
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', return_train_score=True)
    grid_search.fit(X, y)
        # Print the best hyperparameters and the best score
    GridSearchCVresults['Classifiers'].append(clf.__class__.__name__)
    GridSearchCVresults['Params'].append(grid_search.best_params_)
    GridSearchCVresults['BestScore'].append(grid_search.best_score_)
    GridSearchCVresults['BestEstimator'].append(grid_search.best_estimator_)

    print(f"Best hyperparameters for {clf.__class__.__name__}: {grid_search.best_params_}")
    print(f"Best score for {clf.__class__.__name__}: {grid_search.best_score_:.3f}")

GridSearchCVresultsdf = pd.DataFrame(GridSearchCVresults)



In [ ]:
accuracy_score_df

In [ ]:

import pandas as pd
names = [
    
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "Naive Bayes"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(kernel='rbf', random_state=1, gamma=0.01, C=15.0),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=2),
    MLPClassifier(alpha=1, max_iter=1000),
    GaussianNB()
    ]

# Create an empty dataframe
df = pd.DataFrame(columns=['Classifier', 'Parameters'])

# Iterate over the list of classifiers
for name, clf in zip(names, classifiers):
  # Get the classifier's parameters
  params = clf.get_params()
  # Add the classifier name and parameters to the dataframe
  df = df.append({'Classifier': name, 'Parameters': params}, ignore_index=True)

# Display the resulting dataframe
df

In [ ]:

  X, y = X_to_drop, y
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11, test_size=.20)

     
  clf.fit(X_train, y_train)
     
  pred = clf.predict(X_test)
  score = accuracy_score(y_test, pred) 
  cm = confusion_matrix(y_test, pred)
  cr = classification_report(y_test, pred)

  accuracy_scoreDict['Classifier'].append(name)
  accuracy_scoreDict['Accuracy Score'].append(score) 
     
  print("-"*90)
  print("Test Score: {} With Classifier: {}".format(score, clf))
  print("-"*90)
  print("-"*90)
  print("CLASS_REPORT: {}".format(cr))
  print("-"*90)
     
pd.DataFrame(accuracy_scoreDict)
accuracy_score_df = pd.DataFrame(accuracy_scoreDict)     